# Part 2

## Code from HW3

In [1]:
import requests
import pymorphy2
import numpy as np
import re
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
morph = pymorphy2.MorphAnalyzer()
def normalize_split_sentence(sentence):
    sentence_dict = defaultdict(int)
    for word in word_tokenize(re.sub(r'[^\w]|[0-9]+', ' ', sentence, flags=re.U), language='russian'):
        sentence_dict[morph.parse(word)[0].normal_form] += 1
    return sentence_dict

In [3]:
def extract_sentences(urls):
    if type(urls) != list:
        urls = [urls]
    sentences = []
    sentences_normal = []
    for url in urls:
        wiki = requests.get(url)
        soup = BeautifulSoup(wiki.text, 'html.parser')
        content = soup.find(id="mw-content-text")
        for p in content.findAll(['p', 'blockquote', 'ul'], recursive=False):
            for sentence in sent_tokenize(p.text, language='russian'):
                if len(sentence) > 4:
                    sentences.append(sentence)
                    sentences_normal.append(normalize_split_sentence(sentence))
    return np.array(sentences), sentences_normal

In [4]:
def vectorize(sentences_normal, with_idf=True):
    dict_to_vec = DictVectorizer()
    vec_to_weights = TfidfTransformer(use_idf=with_idf)
    sentences_weights = vec_to_weights.fit_transform(dict_to_vec.fit_transform(sentences_normal)).toarray()
    vectorizer = lambda x: vec_to_weights.transform(dict_to_vec.transform(x)).toarray()
    return sentences_weights, vectorizer

In [5]:
def sort_sentences_by_query(urls, query, with_idf=True):
    sentences, sentences_normal = extract_sentences(urls)
    sentences_weights, vectorizer = vectorize(sentences_normal, with_idf=with_idf)
    query_weights = vectorizer(normalize_split_sentence(query))
    cosine = np.sum(sentences_weights*query_weights, axis=1)
    return sentences[cosine.argsort()[::-1]], cosine[cosine.argsort()[::-1]]

In [6]:
def print_sorted(sentences, cosine):
    for i in xrange(len(sentences)):
        print u'{0:2d} [weight={1:7.5f}]: {2}'.format(i+1, cosine[i], sentences[i])

## Code from HW4

In [7]:
def DCG(scale):
    scale = np.array(scale).astype(float)
    scale[1:] = scale[1:] / np.log2(np.arange(2, len(scale)+1))
    return scale.sum()

In [8]:
def IDCG(scale):
    return DCG(np.sort(scale)[::-1])

In [9]:
def nDCG(scale):
    return DCG(scale)/IDCG(scale)

# Задание

## Запрос 1: Мишель Обама и Пэн Лиюань назвали сына знаменитой панды (на илл.) «драгоценным сокровищем».

In [10]:
url = u'https://ru.wikipedia.org/wiki/Мэй_Сян'
query = u'Мишель Обама и Пэн Лиюань назвали сына знаменитой панды (на илл.) «драгоценным сокровищем».'

### Разметим предложения

In [11]:
sentences, _ = extract_sentences(url)

marks = defaultdict(int)
marks[sentences[18]] = 2
marks[sentences[19]] = 2
marks[sentences[20]] = 2
marks[sentences[17]] = 1

# Лучший порядок:

In [12]:
best_order = [18, 19, 20, 17]
for i in best_order:
    print u'{0}:\t{1}'.format(marks[sentences[i]], sentences[i])
print u'...(остальные предложения)'

2:	Первые леди раскрыли свитки с именем медведя — Обама показала присутствующим написание имени на английском языке, а Пэн Лиюань — на китайском[6].
2:	Первые леди назвали панду «Бэй Бэй».
2:	Имя Бэй Бэй переводится как «драгоценное сокровище».
1:	25 сентября 2015 года малыша навестили первая леди США, Мишель Обама, и первая леди Китая Пэн Люиянь.
...(остальные предложения)


## Оценим системы с IDF и без IDF

In [13]:
sentences_sorted, cosine = sort_sentences_by_query(url, query, with_idf=True)
s_idf = [marks[sent] for sent in sentences_sorted]

In [14]:
sentences_sorted, cosine = sort_sentences_by_query(url, query, with_idf=False)
s_no_idf = [marks[sent] for sent in sentences_sorted]

In [15]:
print 'System with IDF: {0}'.format(nDCG(s_idf))
print 'System without IDF: {0}'.format(nDCG(s_no_idf))
print 'Average nDCG: {0}'.format(np.mean([nDCG(x) for x in [s_idf, s_no_idf]]))

System with IDF: 0.913222458934
System without IDF: 0.804437998683
Average nDCG: 0.858830228808


## Запрос 2: Создатель бело-красно-белого флага (на илл.) сидел в немецких и советских лагерях.

In [16]:
url = u'https://ru.wikipedia.org/wiki/Дуж-Душевский,_Клавдий_Степанович'
query = u'Создатель бело-красно-белого флага (на илл.) сидел в немецких и советских лагерях.'

### Разметим предложения

In [17]:
sentences, _ = extract_sentences(url)

In [18]:
marks = defaultdict(int)
marks[sentences[2]] = 1
marks[sentences[28]] = 2
marks[sentences[29]] = 2
marks[sentences[30]] = 1
marks[sentences[31]] = 1

# Лучший порядок:

In [19]:
best_order = [28, 29, 30, 31, 2]
for i in best_order:
    print u'{0}:\t{1}'.format(marks[sentences[i]], sentences[i])
print u'...(остальные предложения)'

2:	После оккупации Литовской ССР немецкой армией отказался от сотрудничества с немцами, за что (а также за помощь евреям) в 1943 году попал в концлагерь[3].
2:	После освобождения Литвы короткое время работал доцентом Каунасского университета, но в 1946 году вновь был арестован советскими властями.
1:	Был приговорён к 25 годам заключения с конфискацией имущества, однако затем срок был сокращён до 10 лет.
1:	По состоянию здоровья Дуж-Душевский досрочно был освобождён из тюрьмы в 1955 году.
1:	Считается автором[1][2] бело-красно-белого флага — официального флага Белорусской народной республики (1918—1919) и Республики Беларусь (1991—1995).
...(остальные предложения)


## Оценим системы с IDF и без IDF

In [20]:
sentences_sorted, cosine = sort_sentences_by_query(url, query, with_idf=True)
s_idf = [marks[sent] for sent in sentences_sorted]

In [21]:
sentences_sorted, cosine = sort_sentences_by_query(url, query, with_idf=False)
s_no_idf = [marks[sent] for sent in sentences_sorted]

In [22]:
print 'System with IDF: {0}'.format(nDCG(s_idf))
print 'System without IDF: {0}'.format(nDCG(s_no_idf))
print 'Average nDCG: {0}'.format(np.mean([nDCG(x) for x in [s_idf, s_no_idf]]))

System with IDF: 0.765318519438
System without IDF: 0.444545686425
Average nDCG: 0.604932102931


## Запрос 3: Минирование берегов спасает популяцию пингвинов.

In [23]:
url = u'https://ru.wikipedia.org/wiki/FMK-3_(мина)'
query = u'Минирование берегов спасает популяцию пингвинов.'

### Разметим предложения

In [24]:
sentences, _ = extract_sentences(url)

In [25]:
marks = defaultdict(int)
marks[sentences[12]] = 2
marks[sentences[13]] = 1
marks[sentences[14]] = 2

# Лучший порядок:

In [26]:
best_order = [12, 14, 13]
for i in best_order:
    print u'{0}:\t{1}'.format(marks[sentences[i]], sentences[i])
print u'...(остальные предложения)'

2:	Эти огороженные колючей проволокой территории неожиданно способствовали увеличению популяции пингвинов, находившихся здесь на грани исчезновения[10].
2:	Таким образом, они могут благополучно размножаться без помех со стороны людей.
1:	Вес пингвина оказался недостачным для срабатывания мин.
...(остальные предложения)


## Оценим системы с IDF и без IDF

In [27]:
sentences_sorted, cosine = sort_sentences_by_query(url, query, with_idf=True)
s_idf = [marks[sent] for sent in sentences_sorted]

In [28]:
print_sorted(sentences_sorted[:5], cosine[:5])

 1 [weight=0.28399]: Эти огороженные колючей проволокой территории неожиданно способствовали увеличению популяции пингвинов, находившихся здесь на грани исчезновения[10].
 2 [weight=0.25425]: В. Минное оружие: вопросы минирования и разминирования.
 3 [weight=0.20588]: Вес пингвина оказался недостачным для срабатывания мин.
 4 [weight=0.00000]: Разработана в Аргентине.
 5 [weight=0.00000]: Производство велось государственной оборонной компанией DGFM (англ.)русск..


In [29]:
sentences_sorted, cosine = sort_sentences_by_query(url, query, with_idf=False)
s_no_idf = [marks[sent] for sent in sentences_sorted]

In [30]:
print_sorted(sentences_sorted[:5], cosine[:5])

 1 [weight=0.29814]: Эти огороженные колючей проволокой территории неожиданно способствовали увеличению популяции пингвинов, находившихся здесь на грани исчезновения[10].
 2 [weight=0.21822]: Вес пингвина оказался недостачным для срабатывания мин.
 3 [weight=0.21822]: В. Минное оружие: вопросы минирования и разминирования.
 4 [weight=0.00000]: Разработана в Аргентине.
 5 [weight=0.00000]: Производство велось государственной оборонной компанией DGFM (англ.)русск..


In [31]:
print 'System with IDF: {0}'.format(nDCG(s_idf))
print 'System without IDF: {0}'.format(nDCG(s_no_idf))
print 'Average nDCG: {0}'.format(np.mean([nDCG(x) for x in [s_idf, s_no_idf]]))

System with IDF: 0.673780645323
System without IDF: 0.753477437559
Average nDCG: 0.713629041441


# Вывод

### Обычно система с IDF показывает лучший результат, но в последнем примере видно, что система без IDF показазала себя лучше. Связано это с тем, что запрос никак не расширялся и для поиска использовались только формы слов из запроса. В запросе попались низкочастотные слова (минирование)  и предожение с этим словом вышло на 2-е место в IDF-системе, хотя оно несет меньше полезной информации по запросу. (В защиту IDF можно сказать, что DF присвоила одинаковый вес этому предложение и размеченному, которое в IDF оказалось ниже).